# Processamento de Lançamentos de Funding e Aplicações

## Objtetivo

<li> Atribuir origens e aplicações as carteiras seguindo a priorização das parametrizações do plano funding
<li> Efetivar lançamentos de custos e receitas de oportunidade as operações aos recursos cedidos/aplicados
<li> Apurar a taxa de oportunidade de cada recurso no caixa central para levar os custos e receitas para os PA's

## Import Library

In [1]:
# bibliotecas de manipulação
import pandas as pd
import numpy as np

# bibliotecas de visualização
import matplotlib as plt
import seaborn as sns

# bibliotecas de machine learning

# outras bibliotecas
import datetime
import os
import warnings

In [2]:
# opções para o pandas
pd.options.display.max_columns=None
pd.options.display.max_rows=None
pd.options.mode.use_inf_as_na=True
pd.options.display.max_colwidth=None

# opções para visualizações
%matplotlib inline

# opções para Jupyter notebook
warnings.filterwarnings('ignore')

## Importação de bibliotecas ou classes locais

## Definição de Parâmetros

In [3]:
# caminho do plano funding definitivo com os parâmetros
path_output_plano='../data/output/plano/plano-funding-definitivo.xlsx'

# caminho do balanço funding
path_output_balanco='../data/output/balanco/balanco-funding.xlsx'

# caminho da base de indicadores
path_input_indicadores='../data/input/plano/plano-funding-pad.xlsx'

# lista com a ordem de uso de origens e aplicações
list_ordem_funding=['NumOrdemUtilizacaoMoeda', 'OrdemUtilizacaoFunding']

metricas=['SMDC', 'SF', 'VlrJuros']

## Definição de funções

In [4]:
def obter_valor_indicador(param_data, param_ctafunding, param_codmoeda, param_codindicador):
    # ano mês de apuração do indicador
    vlrAnoMes=str(param_data)[:7]

    # código do indicador de transferência
    vlrCodTransferencia=param_codindicador
    
    # código da conta funding
    vlrCtaFunding=param_ctafunding
    
    # código da moeda
    vlrCodMoeda=param_codmoeda
    
    # capturando valor apurado do indicador no mês
    vlrIndicador=df_indicadores[
        (df_indicadores.AnoMes==vlrAnoMes) &
        (df_indicadores.CodPrecoTransferencia==vlrCodTransferencia)
    ].Valor.values[0]
    
    # capturando a natureza do recurso para corrigir valor
    #vlrNaturezaAplicacao=df_plano[
    #    (df_plano.CtaFunding==vlrCtaFunding) &
    #    (df_plano.CodMoeda==vlrCodMoeda)
    #].BolAplicacao.values[0]
    
    # se a conta for de origem, corrigir o valor do indicador para negativo
    #if vlrNaturezaAplicacao==0:
    #    vlrIndicador = -vlrIndicador
    #else:
    #    pass
    
    # retornando o valor de apuração do indicador de transferência de preços no mês de referência
    return vlrIndicador


# retorna a taxa de oportunidade do caixa central sintético
def obter_taxa_oportunidade(param_coop, param_data, param_ctafunding, param_moeda):
    
    try:
        vlrTaxa=df_caixacentral[
            (df_caixacentral.NumCooperativa==param_coop) &
            (df_caixacentral.DtMovimento==param_data) &
            (df_caixacentral.CtaFunding==param_ctafunding) &
            (df_caixacentral.CodMoeda==param_moeda)
        ].TaxaOportunidade.values[0]
    
    except:
        vlrTaxa=0.0
    
    return vlrTaxa

## Importação de Dados

In [5]:
# importa o balanço funding
df_balanco_padrao=pd.read_excel(path_output_balanco, sheet_name='funding', header=0, index_col=0)
# em função das contas de duplo posicionamento 'DC', atribuindo marcação se a conta é de origem ou aplicação
df_balanco_padrao['BolOrigem']=df_balanco_padrao['SMDC'].apply(lambda x: 1 if x >= 1 else 0)
df_balanco_padrao.sort_values(by=['NumCooperativa', 'DtMovimento'], inplace=True)
df_balanco=df_balanco_padrao.copy()
df_balanco=df_balanco.drop(columns=['NumPA','VlrTaxaJuros'])

# preenchendo dados nulos para agregar métricas
df_balanco.CtaDesdobramento.fillna(-1, inplace=True)

# agrupar balanço removendo PA
df_balanco=df_balanco.groupby(by=[i for i in df_balanco.columns if i not in metricas])[metricas].sum().reset_index()

# calculando taxa média de juros do recurso/aplicação em nível de cooperativa
df_balanco['VlrTaxaJurosFunding']=df_balanco.VlrJuros/df_balanco.SMDC
df_balanco.VlrTaxaJurosFunding.fillna(0,inplace=True)

# importa o plano funding
df_plano=pd.read_excel(path_output_plano, sheet_name='plano', header=0, index_col=0)
# excluindo contas de desdobramento do plano
df_plano=df_plano[df_plano.BolDesdobrarConta!=1]
df_plano.CodMoeda=df_plano.CodMoeda.astype('int64')
df_plano.CodPrecoTransferencia=df_plano.CodPrecoTransferencia.astype('int64')

# importa a base de apuração de indicadores
df_indicadores=pd.read_excel(path_input_indicadores, sheet_name='PrecosTransferenciaApurados', header=0)
df_indicadores.Valor=df_indicadores.Valor/100

## Processamento lançamentos analíticos do caixa central

In [6]:
# criando dataframe para receber os lançamentos
df_lancamento=pd.DataFrame(
    {
        'NumCooperativa': pd.Series([], dtype='int'),
        'DtMovimento': pd.Series([], dtype='datetime64[ns]'),
        'NumOrdemOrigem': pd.Series([], dtype='int'),
        'NumOrdemAplicacao': pd.Series([], dtype='int'),
        'CtaFundingOrigem': pd.Series([], dtype='int'),
        'CodMoedaOrigem': pd.Series([], dtype='object'),
        'CtaFundingAplicacao': pd.Series([], dtype='int'),
        'CodMoedaAplicacao': pd.Series([], dtype='object'),
        'CodPrecoTransferencia': pd.Series([], dtype='int'),
        'VlrTaxaOportunidade': pd.Series([],dtype='float64'),
        'SmdcOrigem': pd.Series([], dtype='float64'),
        'SmdcAplicacao': pd.Series([], dtype='float64'),
        'VlrJurosOrigem': pd.Series([], dtype='float64'),
        'VlrJurosAplicacao': pd.Series([], dtype='float64'),
        'VlrReceitaOportunidade': pd.Series([], dtype='float64'),
        'VlrCustoOportunidade': pd.Series([],dtype='float64')
    }
)

# capturando períodos a serem processados
i=df_balanco.groupby(by=['NumCooperativa', 'DtMovimento']).size().reset_index()[['NumCooperativa', 'DtMovimento']]
periodo=list(zip(i.NumCooperativa, i.DtMovimento))



# processando cooperativa a cooperativa, mês a mês
for coop, mes in periodo:
    
    # atribuindo origens a serem iteradas
    df_origem=df_balanco[
        (df_balanco.BolOrigem==1) &
        (df_balanco.NumCooperativa==coop) &
        (df_balanco.DtMovimento==mes)
    ].sort_values(by=list_ordem_funding)[
    ['NumCooperativa', 'DtMovimento', 'CtaFunding', 'CodMoeda', 'SMDC', 'VlrJuros', 'VlrTaxaJurosFunding']
    ].reset_index(drop=True)

    
    # atribuindo código dos preços de transferência a serem usados na precificação
    df_origem=df_origem.merge(
        df_plano[['CtaFunding','CodMoeda','CodPrecoTransferencia','NomePrecoTransferencia']],
        how='inner',
        left_on=['CtaFunding', 'CodMoeda'],
        right_on=['CtaFunding', 'CodMoeda']
    )

    
    # capturando o valor do preço de transferência para o código de moeda apurado no mês
    df_origem['VlrTaxaJurosOportunidade']=df_origem.apply(lambda x:
                                                          obter_valor_indicador(
                                                              x.DtMovimento, x.CtaFunding, x.CodMoeda, x.CodPrecoTransferencia
                                                              ), axis=1)
       
    # atribuindo aplicações a serem iteradas
    df_aplicacao=df_balanco[
        (df_balanco.BolOrigem==0) &
        (df_balanco.NumCooperativa==coop) &
        (df_balanco.DtMovimento==mes)
    ].sort_values(by=list_ordem_funding)[
        ['NumCooperativa', 'DtMovimento', 'CtaFunding', 'CodMoeda', 'SMDC', 'VlrJuros', 'VlrTaxaJurosFunding']
    ].reset_index(drop=True)
    
    
    # iterando pelo dataframe de origens
    for num_origem in range(df_origem.shape[0]):
        
        # atribuindo dicionário da origem iterada
        dic_origem=df_origem.iloc[num_origem].to_dict()
        
        # atribuindo número de ordem da origem ao dicionário
        dic_origem['NumOrdemOrigem']=num_origem+1
        
        
        # iterando pelo dataframe de aplicações:
        for num_aplicacao in range(df_aplicacao.shape[0]):
            
            # atribuindo dicionário da aplicação iterada
            dic_aplicacao=df_aplicacao.iloc[num_aplicacao].to_dict()
            
            # atribuindo número de ordem da aplicação ao dicionário
            dic_aplicacao['NumOrdemAplicacao']=num_aplicacao+1
            
            # atribuindo saldo da origem já mapeada
            dic_origem['SMDCAcumulado']=df_lancamento[
                (df_lancamento.NumCooperativa==coop) &
                (df_lancamento.DtMovimento==mes) &
                (df_lancamento.CtaFundingOrigem==dic_origem['CtaFunding']) &
                (df_lancamento.CodMoedaOrigem==dic_origem['CodMoeda'])
                ].SmdcOrigem.sum()
            
            # atribuindo saldo da aplicacao já mapeada
            dic_aplicacao['SMDCAcumulado']=df_lancamento[
                (df_lancamento.NumCooperativa==coop) &
                (df_lancamento.DtMovimento==mes) &
                (df_lancamento.CtaFundingAplicacao==dic_aplicacao['CtaFunding']) &
                (df_lancamento.CodMoedaAplicacao==dic_aplicacao['CodMoeda'])
                ].SmdcAplicacao.sum()
            
            
            if dic_origem['SMDCAcumulado']==dic_origem['SMDC']:
                
                # se o valor da origem já estiver totalmente mapeada, sair da iteração para a próxima origem
                break
            else:
                
                if dic_aplicacao['SMDCAcumulado']==dic_aplicacao['SMDC']:
                    
                    # se o valor da aplicação já estiver totalmente mapeado, passar para a próxima aplicação
                    pass
                else:
                    
                    # atribuindo saldo remanescente de origem
                    vlrSaldoOrigem=dic_origem['SMDC']-dic_origem['SMDCAcumulado']
                    
                    # atribuindo saldo remanescente de aplicação
                    vlrSaldoAplicacao=dic_aplicacao['SMDC']-dic_aplicacao['SMDCAcumulado']
                    
                    # calculando o valor do mapeamento
                    if abs(vlrSaldoAplicacao) <= abs(vlrSaldoOrigem):
                        vlrAplicacao=vlrSaldoAplicacao

                    else:
                        vlrAplicacao=-vlrSaldoOrigem
                        
                    vlrOrigem=abs(vlrAplicacao)
                    
                    # atribuindo juros proporcional ao valor mapeado
                    # transferindo juros reais entre operações mapeadas
                    vlrJurosAplicacao=vlrAplicacao * dic_aplicacao['VlrTaxaJurosFunding']
                    vlrJurosOrigem=vlrOrigem * dic_origem['VlrTaxaJurosFunding']
                    
                    # atribuindo taxas de oportunidade
                    vlrTaxaOportunidade=dic_origem['VlrTaxaJurosOportunidade']
                    
                    # atribuindo custo e receita de oportunidade entre operações mapeadas
                    vlrCustoOportunidade=vlrAplicacao * vlrTaxaOportunidade
                    vlrReceitaOportunidade=vlrOrigem * vlrTaxaOportunidade
                    
                    # inserindo dados no dataframe de lançamento
                    df_lancamento=df_lancamento.append(
                        pd.DataFrame(
                            [
                                coop, 
                                mes, 
                                dic_origem['NumOrdemOrigem'], 
                                dic_aplicacao['NumOrdemAplicacao'],
                                dic_origem['CtaFunding'],
                                dic_origem['CodMoeda'],
                                dic_aplicacao['CtaFunding'],
                                dic_aplicacao['CodMoeda'],
                                dic_origem['CodPrecoTransferencia'],
                                vlrTaxaOportunidade,
                                vlrOrigem,
                                vlrAplicacao,
                                vlrJurosOrigem,
                                vlrJurosAplicacao,
                                vlrReceitaOportunidade,
                                vlrCustoOportunidade,
                            ],index=['NumCooperativa', 'DtMovimento', 'NumOrdemOrigem', 'NumOrdemAplicacao', 'CtaFundingOrigem', 
                                     'CodMoedaOrigem', 'CtaFundingAplicacao','CodMoedaAplicacao', 'CodPrecoTransferencia',
                                     'VlrTaxaOportunidade','SmdcOrigem', 'SmdcAplicacao', 'VlrJurosOrigem', 'VlrJurosAplicacao', 
                                     'VlrReceitaOportunidade', 'VlrCustoOportunidade']
                            ).T
                        )

df_lancamento.reset_index(drop=True, inplace=True)


# acrescentando métricas financeiras das funções de origem e aplicação
df_lancamento['MargemFinanceiraBruta']=df_lancamento.VlrJurosOrigem + df_lancamento.VlrJurosAplicacao
df_lancamento['MargemFinanceiraOrigem']=df_lancamento.VlrReceitaOportunidade + df_lancamento.VlrJurosOrigem
df_lancamento['MargemFinanceiraAplicacao']=df_lancamento.VlrJurosAplicacao + df_lancamento.VlrCustoOportunidade

## Processando cálculo das taxas de oportunidade do caixa central

In [7]:
df_caixacentral=pd.DataFrame()


# acrescentando origens
df_caixacentral=df_caixacentral.append(df_lancamento.groupby(by=[
    'NumCooperativa', 'DtMovimento', 'CtaFundingOrigem', 'CodMoedaOrigem'
])['SmdcOrigem', 'VlrJurosOrigem', 'VlrReceitaOportunidade', 'MargemFinanceiraOrigem'].sum().reset_index().rename(columns={
    'CtaFundingOrigem':'CtaFunding',
    'CodMoedaOrigem': 'CodMoeda',
    'SmdcOrigem': 'SMDC',
    'VlrJurosOrigem': 'VlrJuros',
    'VlrReceitaOportunidade': 'VlrOportunidade',
    'MargemFinanceiraOrigem': 'VlrMargemFinanceiraBruta'
}))

# acrescentando aplicações
df_caixacentral=df_caixacentral.append(df_lancamento.groupby(by=[
    'NumCooperativa', 'DtMovimento', 'CtaFundingAplicacao', 'CodMoedaAplicacao'
])['SmdcAplicacao', 'VlrJurosAplicacao','VlrCustoOportunidade','MargemFinanceiraAplicacao'].sum().reset_index().rename(columns={
    'CtaFundingAplicacao':'CtaFunding',
    'CodMoedaAplicacao': 'CodMoeda',
    'SmdcAplicacao': 'SMDC',
    'VlrJurosAplicacao': 'VlrJuros',
    'VlrCustoOportunidade': 'VlrOportunidade',
    'MargemFinanceiraAplicacao': 'VlrMargemFinanceiraBruta'
}))


df_caixacentral.reset_index(inplace=True, drop=True)

# calculando taxas
df_caixacentral['TaxaOportunidade']=df_caixacentral.VlrOportunidade/df_caixacentral.SMDC
df_caixacentral.sort_values(by=['NumCooperativa', 'DtMovimento', 'CtaFunding', 'CodMoeda'], inplace=True)

## Atribuindo custo e receita de oportunidade ao PA

In [8]:
df_balanco_pa=df_balanco_padrao.copy()

df_balanco_pa['VlrJurosOportunidade']=df_balanco_pa.apply(
    lambda x: obter_taxa_oportunidade(
        x.NumCooperativa, x.DtMovimento, x.CtaFunding, x.CodMoeda
    ) * x.SMDC, axis=1
)

## Acrescentando atributos dos planos aos dataframes a serem exportados

In [9]:
# acrescentando atributos ao dataframe de caixa central analítico - df_lancamento

# origens
df_lancamento=df_lancamento.merge(
    df_plano[['CtaFunding', 'Nome','CodMoeda','NomeMoeda', 'Origem', 'TipoMoeda','VlrPrazoFinal']],
    how='inner',
    left_on=['CtaFundingOrigem', 'CodMoedaOrigem'],
    right_on=['CtaFunding', 'CodMoeda']

).rename(
    columns={'Nome':'NomeFundingOrigem', 'NomeMoeda':'NomeMoedaOrigem', 'TipoMoeda': 'TipoMoedaOrigem',
             'Origem':'TipoOrigem', 'VlrPrazoFinal':'VlrPrazoFinalOrigem'}
).drop(columns=['CodMoeda','CtaFunding'])


# aplicações
df_lancamento=df_lancamento.merge(
    df_plano[['CtaFunding', 'Nome','CodMoeda','NomeMoeda', 'Origem', 'TipoMoeda','VlrPrazoFinal']],
    how='inner',
    left_on=['CtaFundingAplicacao', 'CodMoedaAplicacao'],
    right_on=['CtaFunding', 'CodMoeda']

).rename(
    columns={'Nome':'NomeFundingAplicacao', 'NomeMoeda':'NomeMoedaAplicacao', 'TipoMoeda': 'TipoMoedaAplicacao', 
             'Origem': 'TipoAplicacao','VlrPrazoFinal': 'VlrPrazoFinalAplicacao'}
).drop(columns=['CodMoeda','CtaFunding'])

metricas=[
    'CodPrecoTransferencia',
    'VlrTaxaOportunidade',
    'SmdcOrigem',
    'SmdcAplicacao',
    'VlrJurosOrigem',
    'VlrJurosAplicacao',
    'VlrReceitaOportunidade',
    'VlrCustoOportunidade',
    'MargemFinanceiraBruta',
    'MargemFinanceiraOrigem',
    'MargemFinanceiraAplicacao'
]

df_lancamento=df_lancamento[[i for i in df_lancamento.columns if i not in metricas] + metricas]


# acrescentando atributos ao dataframe de caixa central sintético - df_caixacentral
df_caixacentral=df_caixacentral.merge(
    df_plano[['CtaFunding', 'Nome','CodMoeda','NomeMoeda', 'Origem', 'TipoMoeda','VlrPrazoFinal']],
    how='inner',
    left_on=['CtaFunding', 'CodMoeda'],
    right_on=['CtaFunding', 'CodMoeda']

).rename(columns={'Nome':'NomeFunding'})

metricas=[
    'SMDC',
    'VlrJuros',
    'VlrOportunidade',
    'VlrMargemFinanceiraBruta',
    'TaxaOportunidade'
]

df_caixacentral=df_caixacentral[[i for i in df_caixacentral.columns if i not in metricas] + metricas]

## Output

In [10]:
# salvar arquivo caixa central analítico
df_lancamento.to_excel('../data/output/lancamento/caixa-central-analitico.xlsx', sheet_name='analitico', freeze_panes=(1,1))
# salvar arquivo caixa central sintético
df_caixacentral.to_excel('../data/output/lancamento/caixa-central-sintetico.xlsx', sheet_name='sintetico', freeze_panes=(1,1))
# salvar arquivo balanço com custo oportunidade atribuido
df_balanco_pa.to_excel('../data/output/balanco/balanco-funding-pa.xlsx', sheet_name='balanco', freeze_panes=(1,1))
